# Load Dataset

In [1]:
from google.colab import drive
drive.mount('/content/MyDrive')

Mounted at /content/MyDrive


In [ ]:
!unzip -q "/content/MyDrive/MyDrive/UMHackathon/UMHackathon 2021 Finance Dataset.zip" -d "dataset" 

replace dataset/BPAM Evaluated Prices Rated Range Prices (LT)/201907/BPAMERP20190731.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: N


In [3]:
!pip install selenium -q
!apt-get update 
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin

     |████████████████████████████████| 904 kB 8.1 MB/s 
Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:3 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Get:8 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ Packages [66.6 kB]
Hit:9 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:12 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:14 http://ppa.launchpad.net/

In [4]:
import pandas as pd
import glob
import os
import dateutil.parser
import functools
import matplotlib.pyplot as plt
from statsmodels.graphics.tsaplots import plot_acf
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import numpy as np
import datetime
import random

import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import Select
from dateutil.relativedelta import relativedelta
import requests
from bs4 import BeautifulSoup

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


# Data Cleaning

In [5]:
dataset_dir = "/content/dataset/BPAM Evaluated Prices Rated Range Prices (LT)"

# Merge both files (BPANERP & BPAMERS) together as a single pandas frame
def read_all_csvs(dataset_dir):
  sub_dirs = [f for f in os.listdir(dataset_dir) if not os.path.isfile(f)]
  df_BPAMERP = pd.DataFrame()
  df_BPAMERS = pd.DataFrame()
  df_joined = pd.DataFrame()
  for sub_dir in sub_dirs:
    full_path = os.path.join(dataset_dir, sub_dir)
    for file_name in os.listdir(full_path):
      full_file_name = os.path.join(full_path, file_name)
      if "BPAMERP" in file_name:
        temp_df_BPAMERP =  pd.read_csv(full_file_name, parse_dates=True)
        df_BPAMERP = pd.concat([df_BPAMERP, temp_df_BPAMERP])
      elif "BPAMERS" in file_name:
        temp_df_BPAMERS =  pd.read_csv(full_file_name, parse_dates=True)
        df_BPAMERS = pd.concat([df_BPAMERS, temp_df_BPAMERS])
    df_joined = pd.concat([df_joined, temp_df_BPAMERP.merge(temp_df_BPAMERS, on = ['STOCK CODE','ISIN CODE','STOCK NAME'], how = 'inner')])
  return df_BPAMERP.reset_index(drop=True), df_BPAMERS.reset_index(drop=True), df_joined.sort_values(by=['VALUE DATE']).reset_index(drop=True)

df_BPAMERP, df_BPAMERS, df_joined = read_all_csvs(dataset_dir)
df_joined.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34206 entries, 0 to 34205
Data columns (total 49 columns):
 #   Column                                    Non-Null Count  Dtype  
---  ------                                    --------------  -----  
 0   STOCK CODE                                34206 non-null  object 
 1   ISIN CODE                                 34206 non-null  object 
 2   STOCK NAME                                34206 non-null  object 
 3   VALUE DATE                                34206 non-null  object 
 4   EVAL UPPER THRESHOLD YIELD                34206 non-null  float64
 5   EVAL MID YIELD                            34206 non-null  float64
 6   EVAL LOWER THRESHOLD YIELD                34206 non-null  float64
 7   EVAL LOWER THRESHOLD PRICE                34206 non-null  float64
 8   EVAL MID PRICE                            34206 non-null  float64
 9   EVAL UPPER THRESHOLD PRICE                34206 non-null  float64
 10  MODIFIED DURATION                 

In [6]:
# parse datetime data type
date_columns = [column for column in df_joined.columns if "DATE" in column]
for date_column in date_columns:
  df_joined[date_column] = pd.to_datetime(df_joined[date_column], infer_datetime_format=True)  

# parse bool data type
d = {'Y': True, 'N': False}
df_joined["CALLABLE/PUTTABLE"] = df_joined["CALLABLE/PUTTABLE"].map(d)
df_joined["CONVERTIBLE/EXCHANGABLE"] = df_joined["CONVERTIBLE/EXCHANGABLE"].map(d)

# parse category data type
obj_columns = df_joined.select_dtypes(include='object').columns
for obj_column in obj_columns:
  df_joined[obj_column] = df_joined[obj_column].astype('category')

# show the memory usage
df_joined.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34206 entries, 0 to 34205
Data columns (total 49 columns):
 #   Column                                    Non-Null Count  Dtype         
---  ------                                    --------------  -----         
 0   STOCK CODE                                34206 non-null  category      
 1   ISIN CODE                                 34206 non-null  category      
 2   STOCK NAME                                34206 non-null  category      
 3   VALUE DATE                                34206 non-null  datetime64[ns]
 4   EVAL UPPER THRESHOLD YIELD                34206 non-null  float64       
 5   EVAL MID YIELD                            34206 non-null  float64       
 6   EVAL LOWER THRESHOLD YIELD                34206 non-null  float64       
 7   EVAL LOWER THRESHOLD PRICE                34206 non-null  float64       
 8   EVAL MID PRICE                            34206 non-null  float64       
 9   EVAL UPPER THRESHOLD PRICE  

In [7]:
# check null value
df_joined.isnull().sum().sort_values()

STOCK CODE                                      0
PRINCIPLE                                       0
BOND TYPE                                       0
BOND CLASS                                      0
CONVERTIBLE/EXCHANGABLE                         0
ISSUE DATE                                      0
MATURITY DATE                                   0
DAY COUNT BASIS                                 0
ISSUER NAME                                     0
FACILITY AMOUNT/FACILITY LIMIT(MYR MIL)         0
BOND ISSUE AMOUNT(MYR MIL)                      0
BOND CURRENT OUTSTANDING AMOUNT(MYR MIL)        0
REMAINING TENURE                                0
ISSUER FACILITY LIMIT(MYR MIL)                  0
ISSUER OUTSTANDING AMOUNT(MYR MIL)              0
SECTOR                                          0
CALLABLE/PUTTABLE                               0
FACILITY OUTSTANDING AMOUNT(MYR MIL)            0
FACILITY CODE                                   0
RATING                                          0


In [8]:
# impute null value
df_joined["COUPON FREQUENCY"].fillna(0, inplace = True)
df_joined["NEXT COUPON RATE"].fillna(0, inplace = True)
df_joined["PREVIOUS COUPON RATE"].fillna(0, inplace = True)

In [9]:
# check null value
df_joined.isnull().sum().sort_values()

STOCK CODE                                      0
BOND CLASS                                      0
CONVERTIBLE/EXCHANGABLE                         0
ISSUE DATE                                      0
MATURITY DATE                                   0
DAY COUNT BASIS                                 0
COUPON FREQUENCY                                0
PREVIOUS COUPON RATE                            0
NEXT COUPON RATE                                0
FACILITY AMOUNT/FACILITY LIMIT(MYR MIL)         0
FACILITY OUTSTANDING AMOUNT(MYR MIL)            0
BOND ISSUE AMOUNT(MYR MIL)                      0
BOND CURRENT OUTSTANDING AMOUNT(MYR MIL)        0
REMAINING TENURE                                0
ISSUER FACILITY LIMIT(MYR MIL)                  0
ISSUER OUTSTANDING AMOUNT(MYR MIL)              0
SECTOR                                          0
CALLABLE/PUTTABLE                               0
BOND TYPE                                       0
PRINCIPLE                                       0


# Features Engineering

In [10]:
# extract month data from [VALUE DATE] 
# eg: VALUE DATE 2020-04-01 refers to March, that is why we need to subtract by 1
df_joined["VALUE DATE MONTH"] = df_joined["VALUE DATE"].dt.to_period('M') - 1
df_joined["VALUE DATE MONTH"]

0        2020-03
1        2020-03
2        2020-03
3        2020-03
4        2020-03
          ...   
34201    2020-04
34202    2020-04
34203    2020-04
34204    2020-04
34205    2020-04
Name: VALUE DATE MONTH, Length: 34206, dtype: period[M]

In [11]:
# calculate maturity duration in days 
df_joined["MATURITY DURATION"] = (df_joined["MATURITY DATE"] - df_joined["ISSUE DATE"]).dt.days
df_joined["MATURITY DURATION"]

0         3653
1         9131
2         9129
3         9129
4         9130
         ...  
34201     9130
34202     9129
34203     9129
34204    10956
34205     1277
Name: MATURITY DURATION, Length: 34206, dtype: int64

In [12]:
# calculate accured interest
# assume ACTACT and ACTBOTH are the same
def cal_accured_interest(bond):
  prev_date = bond["PREVIOUS PAYMENT DATE"] if bond["PREVIOUS PAYMENT DATE"] is not pd.NaT else bond["ISSUE DATE"]
  num_of_days_since_last_coupon_payment = float((bond["VALUE DATE"] - prev_date).days)
  if bond["DAY COUNT BASIS"] in ["ACTACT","ACTBOTH"]:
    payment_period = float((bond["NEXT PAYMENT DATE"] - prev_date).days)
    if bond["COUPON FREQUENCY"] == 0:
      return 0
    accured_interest = 100 * (bond["NEXT COUPON RATE"] / 100) / bond["COUPON FREQUENCY"] * num_of_days_since_last_coupon_payment / payment_period
    return accured_interest
  elif bond["DAY COUNT BASIS"] in ["ACT365"]:
    accured_interest = 100 * (bond["NEXT COUPON RATE"] / 100) * num_of_days_since_last_coupon_payment / 365
    return accured_interest

df_joined["ACCRUED INTEREST"] = df_joined.apply(cal_accured_interest, axis=1)
df_joined["ACCRUED INTEREST"]

0        1.660548
1        0.125753
2        1.823562
3        1.930685
4        0.574247
           ...   
34201    1.048000
34202    2.402630
34203    2.293699
34204    2.462466
34205    0.849863
Name: ACCRUED INTEREST, Length: 34206, dtype: float64

In [13]:
def opr_extract():
  URL = "https://www.bnm.gov.my/opr-decision-and-statement"
  page = requests.get(URL)
  soup = BeautifulSoup(page.content,'lxml')
  content=soup.find_all('td')[4:]
  rates=[]
  dates=[]
  for i in range(len(content)):
    if i%4==0:
      dates.append(str(content[i]).replace("<td>","").replace("</td>",""))
    elif i%2==0:
      rates.append(float(str(content[i]).replace("<td>\n\t\t","").replace("\n\t</td>","")))
    else:
      continue
  i=0
  while i in range(len(rates)-1):
    if rates[i] == rates[i+1]:
        del rates[i]
        del dates[i]
    else:
        i += 1
  OPR_movement={}
  for i in range(len(rates)):
    OPR_movement[dates[i]]=rates[i]
  return OPR_movement

def assign_opr(value_date):
  for date, opr in OPR_movement.items(): 
    date = dateutil.parser.parse(date) 
    if value_date >= date:
      return opr

# set opr movement
OPR_movement = opr_extract()
df_joined["OPR movement"] = df_joined["VALUE DATE"].apply(assign_opr)
df_joined["OPR movement"]

0        2.5
1        2.5
2        2.5
3        2.5
4        2.5
        ... 
34201    2.5
34202    2.5
34203    2.5
34204    2.5
34205    2.5
Name: OPR movement, Length: 34206, dtype: float64

In [14]:
def inflation_rate_extract():
  chrome_options = webdriver.ChromeOptions()
  chrome_options.add_argument('--headless')
  chrome_options.add_argument('--no-sandbox')
  chrome_options.add_argument('--disable-dev-shm-usage')
  chrome_options.add_argument("start-maximized")
  chrome_options.add_experimental_option("excludeSwitches", ["enable-automation"])
  chrome_options.add_experimental_option('useAutomationExtension', False)

  wd = webdriver.Chrome('chromedriver',chrome_options=chrome_options)
  wd.get("https://www.fxempire.com/macro/malaysia/inflation-rate")

  #Most recent three years
  wd.find_element_by_xpath('//div[@class=" css-16ycfp3"]').click()
  wd.find_element_by_xpath("//div[@class=' css-14rzzno-menu']//*[name()='div']").click()

  wd.execute_script("return arguments[0].scrollIntoView(true);", WebDriverWait(wd, 20).until(EC.visibility_of_element_located((By.XPATH, "//div[@class='recharts-wrapper']"))))
  elements = WebDriverWait(wd, 20).until(EC.visibility_of_all_elements_located((By.XPATH, "//div[@class='recharts-wrapper']//*[name()='svg']//*[name()='g' and @class='recharts-layer recharts-bar']//*[name()='g']//*[name()='g']//*[name()='g']")))
  ir=[]
  for element in elements:
    ActionChains(wd).move_to_element(element).perform()
    mouseover = WebDriverWait(wd, 5).until(EC.visibility_of_element_located((By.XPATH, "//div[@class='recharts-wrapper']")))
    ir.append(float(mouseover.text.strip().replace("%","")[-6:].replace("\n","")))
  current_date=datetime.date.today()-relativedelta(days=datetime.date.today().day)
  inflation_dict={}
  for i in range(len(ir)-1,-1,-1):
    inflation_dict[str(current_date)]=ir[i]
    current_date-=relativedelta(days=current_date.day)
  inflation_dict['2021-06-30']=3.4
  inflation_dict['2021-05-31']=4.4
  return inflation_dict

def assign_ir(value_date):
  for date, ir in inflation_dict.items(): 
    date = dateutil.parser.parse(date) 
    if value_date >= date:
      return ir

#set inflation rate
inflation_dict=inflation_rate_extract()
df_joined["INFLATION RATE"]=df_joined["VALUE DATE"].apply(assign_ir)
df_joined

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: DeprecationWarning: use options instead of chrome_options
  # Remove the CWD from sys.path while we load stuff.


,STOCK CODE,ISIN CODE,STOCK NAME,VALUE DATE,EVAL UPPER THRESHOLD YIELD,EVAL MID YIELD,EVAL LOWER THRESHOLD YIELD,EVAL LOWER THRESHOLD PRICE,EVAL MID PRICE,EVAL UPPER THRESHOLD PRICE,MODIFIED DURATION,CONVEXITY,EVAL UPPER THRESHOLD YIELD CHANGE,EVAL MID YIELD CHANGE,EVAL LOWER THRESHOLD YIELD CHANGE,EVAL LOWER THRESHOLD PRICE CHANGE,EVAL MID PRICE CHANGE,EVAL UPPER THRESHOLD PRICE CHANGE,COMPOSITE LIQUIDITY SCORE (T-1),FACILITY CODE,ISSUER NAME,PRINCIPLE,BOND TYPE,BOND CLASS,RATING,RATING AGENCY,ISSUE DATE,MATURITY DATE,EXPECTED MATURITY DATE,DAY COUNT BASIS,COUPON FREQUENCY,FIRST PAYMENT DATE,PREVIOUS PAYMENT DATE,PREVIOUS COUPON RATE,NEXT PAYMENT DATE,NEXT COUPON RATE,FACILITY AMOUNT/FACILITY LIMIT(MYR MIL),FACILITY OUTSTANDING AMOUNT(MYR MIL),BOND ISSUE AMOUNT(MYR MIL),BOND CURRENT OUTSTANDING AMOUNT(MYR MIL),REMAINING TENURE,ISSUER FACILITY LIMIT(MYR MIL),ISSUER OUTSTANDING AMOUNT(MYR MIL),ISLAMIC CONCEPT,SECTOR,RATING EFFECTIVE DATE,CALLABLE/PUTTABLE,CONVERTIBLE/EXCHANGABLE,RATING ACTION,VALUE DATE MONTH,MATURITY DURATION,ACCRUED INTEREST,OPR movement,INFLATION RATE
0,UN190345,MYBUN1903458,GENTING RMTN MTN 3653D 08.11.2029 - Tranche 1,2020-04-01,4.543,4.448,4.353,97.196,97.920,98.650,7.684,71.122,-0.023,-0.024,-0.025,0.175,0.184,0.193,3.0,201900062,Genting RMTN Berhad,CONVENTIONAL,Fixed Rate Bond,Corporate(G),AAA (S),RAM,2019-11-08,2029-11-08,NaT,ACT365,2.0,2020-05-08,NaT,0.00,2020-05-08,4.18,10000,1000,460,460,10Y,10000,1000,NaN,TRADING & SERVICES,2019-09-18,False,False,Initial,2020-03,3653,1.660548,2.5,-0.2
1,VZ170120,MYBVZ1701201,DANAINFRA IMTN 5.100% 21.03.2042 - Tranche No 59,2020-04-01,4.485,4.372,4.259,108.536,110.211,111.922,13.574,248.751,-0.058,-0.060,-0.062,0.846,0.893,0.942,0.0,201200042,DanaInfra Nasional Berhad,ISLAMIC,Fixed Rate Bond,Quasi-Govt,NR(LT),NaN,2017-03-21,2042-03-21,NaT,ACT365,2.0,2017-09-21,2020-03-23,5.10,2020-09-21,5.10,61000,55820,285,285,25Y,74000,63820,MURABAHAH,FINANCIAL SERVICES,NaT,False,False,NaN,2020-03,9131,0.125753,2.5,-0.2
2,VZ170163,MYBVZ1701631,DANAINFRA IMTN 5.200% 23.05.2042 - Tranche No 66,2020-04-01,4.484,4.376,4.268,109.981,111.604,113.260,13.379,246.144,-0.058,-0.060,-0.062,0.858,0.905,0.954,1.7,201200042,DanaInfra Nasional Berhad,ISLAMIC,Fixed Rate Bond,Quasi-Govt,NR(LT),NaN,2017-05-25,2042-05-23,NaT,ACT365,2.0,2017-11-27,2019-11-25,5.20,2020-05-27,5.20,61000,55820,780,780,25Y,74000,63820,MURABAHAH,FINANCIAL SERVICES,NaT,False,False,NaN,2020-03,9129,1.823562,2.5,-0.2
3,VZ170377,MYBVZ1703777,DANAINFRA IMTN 5.220% 14.11.2042 - Tranche No 71,2020-04-01,4.495,4.387,4.279,110.223,111.868,113.546,13.513,252.626,-0.058,-0.060,-0.062,0.870,0.918,0.967,1.7,201200042,DanaInfra Nasional Berhad,ISLAMIC,Fixed Rate Bond,Quasi-Govt,NR(LT),NaN,2017-11-16,2042-11-14,NaT,ACT365,2.0,2018-05-16,2019-11-18,5.22,2020-05-18,5.22,61000,55820,600,600,25Y,74000,63820,MURABAHAH,FINANCIAL SERVICES,NaT,False,False,NaN,2020-03,9129,1.930685,2.5,-0.2
4,VZ180073,MYBVZ1800730,DANAINFRA IMTN 5.240% 20.02.2043 - Tranche No 76,2020-04-01,4.509,4.396,4.283,110.364,112.098,113.868,13.759,259.435,-0.058,-0.060,-0.062,0.875,0.925,0.975,0.0,201200042,DanaInfra Nasional Berhad,ISLAMIC,Fixed Rate Bond,Quasi-Govt,NR(LT),NaN,2018-02-21,2043-02-20,NaT,ACT365,2.0,2018-08-21,2020-02-21,5.24,2020-08-21,5.24,61000,55820,1000,1000,25Y,74000,63820,MURABAHAH,FINANCIAL SERVICES,NaT,False,False,NaN,2020-03,9130,0.574247,2.5,-0.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34201,VZ180073,MYBVZ1800730,DANAINFRA IMTN 5.240% 20.02.2043 - Tranche No 76,2020-05-04,3.895,3.781,3.667,120.194,122.153,124.156,14.101,268.667,-0.013,-0.012,-0.011,0.215,0.201,0.188,0.0,201200042,DanaInfra Nasional Berhad,ISLAMIC,Fixed Rate Bond,Quasi-Govt,NR(LT),NaN,2018-02-21,2043-02-20,NaT,ACT365,2.0,2018-08-21,2020-02-21,5.24,2020-08-21,5.24,61000,56820,1000,1000,25Y,74000,64820,MUR

In [15]:
def mgs_extract():
  value_dates=df_joined["VALUE DATE"].unique()
  conv_MGS_movement={}
  islam_MGS_movement={}
  for date in value_dates:
    date=str(date)[:10]
    prev_date=datetime.date(int(date[0:4]),int(date[5:7]),int(date[8:10]))-relativedelta(days=1)
    URL = "https://www.bnm.gov.my/government-securities-yield?p_p_id=my_gov_bnm_yield_display_portlet&p_p_lifecycle=0&p_p_state=normal&p_p_mode=view&_my_gov_bnm_yield_display_portlet_tradingDateTxt="+str(prev_date)
    page = requests.get(URL)
    soup = BeautifulSoup(page.content,'html.parser')
    dates=[]
    mgs=[]
    content=soup.find_all('td')[11:43]
    #Get from earlier dates in the case of weekends/public holidays
    while(str(content[5]).strip().replace('<td align="center">\n','').replace('</td>','').replace('\n','').replace(' ','')==''):
      prev_date-=relativedelta(days=1)
      URL = "https://www.bnm.gov.my/government-securities-yield?p_p_id=my_gov_bnm_yield_display_portlet&p_p_lifecycle=0&p_p_state=normal&p_p_mode=view&_my_gov_bnm_yield_display_portlet_tradingDateTxt="+str(prev_date)
      page = requests.get(URL)
      soup = BeautifulSoup(page.content,'html.parser')
      content=soup.find_all('td')[11:43]
    for i in range(len(content)):
      if i%8==5:
        mgs.append(str(content[i]).strip().replace('<td align="center">\n','').replace('</td>','').replace('\n','').replace(' ',''))
    conv_MGS_movement[date]={}
    conv_MGS_movement[date]["3-year"]=mgs[0]
    conv_MGS_movement[date]["5-year"]=mgs[1]
    conv_MGS_movement[date]["7-year"]=mgs[2]
    conv_MGS_movement[date]["10-year"]=mgs[3]
    content=soup.find_all('td')[50:]
    mgs=[]
    for i in range(len(content)):
      if i%7==5:
        mgs.append(str(content[i]).strip().replace('<td align="center">\n','').replace('</td>','').replace('\n','').replace(' ','').replace('*',''))
    islam_MGS_movement[date]={}
    islam_MGS_movement[date]["3-year"]=mgs[0]
    islam_MGS_movement[date]["5-year"]=mgs[1]
    islam_MGS_movement[date]["7-year"]=mgs[2]
    islam_MGS_movement[date]["10-year"]=mgs[3]
  return conv_MGS_movement,islam_MGS_movement

def assign_mgs(islamic,value_date,remaining_tenure):
  date=str(value_date)[:10]
  mgs_dict=conv_MGS_movement
  if islamic:
    mgs_dict=islam_MGS_movement
  if remaining_tenure in ["3M","6M","1Y","2Y","3Y"]:
    return mgs_dict[date]["3-year"]
  elif remaining_tenure=="5Y":
    return mgs_dict[date]["5-year"]
  elif remaining_tenure=="&Y":
    return mgs_dict[date]["7-year"]
  return mgs_dict[date]["10-year"]

#set mgs movement
conv_MGS_movement, islam_MGS_movement=mgs_extract()
df_joined["PRINCIPLE"]=df_joined["PRINCIPLE"].replace("CONVENTIONAL",0).replace("ISLAMIC",1)
df_joined["MGS"]=df_joined[["PRINCIPLE","VALUE DATE","REMAINING TENURE"]].apply(lambda x: assign_mgs(x["PRINCIPLE"],x["VALUE DATE"],x["REMAINING TENURE"]),axis=1)
df_joined

,STOCK CODE,ISIN CODE,STOCK NAME,VALUE DATE,EVAL UPPER THRESHOLD YIELD,EVAL MID YIELD,EVAL LOWER THRESHOLD YIELD,EVAL LOWER THRESHOLD PRICE,EVAL MID PRICE,EVAL UPPER THRESHOLD PRICE,MODIFIED DURATION,CONVEXITY,EVAL UPPER THRESHOLD YIELD CHANGE,EVAL MID YIELD CHANGE,EVAL LOWER THRESHOLD YIELD CHANGE,EVAL LOWER THRESHOLD PRICE CHANGE,EVAL MID PRICE CHANGE,EVAL UPPER THRESHOLD PRICE CHANGE,COMPOSITE LIQUIDITY SCORE (T-1),FACILITY CODE,ISSUER NAME,PRINCIPLE,BOND TYPE,BOND CLASS,RATING,RATING AGENCY,ISSUE DATE,MATURITY DATE,EXPECTED MATURITY DATE,DAY COUNT BASIS,COUPON FREQUENCY,FIRST PAYMENT DATE,PREVIOUS PAYMENT DATE,PREVIOUS COUPON RATE,NEXT PAYMENT DATE,NEXT COUPON RATE,FACILITY AMOUNT/FACILITY LIMIT(MYR MIL),FACILITY OUTSTANDING AMOUNT(MYR MIL),BOND ISSUE AMOUNT(MYR MIL),BOND CURRENT OUTSTANDING AMOUNT(MYR MIL),REMAINING TENURE,ISSUER FACILITY LIMIT(MYR MIL),ISSUER OUTSTANDING AMOUNT(MYR MIL),ISLAMIC CONCEPT,SECTOR,RATING EFFECTIVE DATE,CALLABLE/PUTTABLE,CONVERTIBLE/EXCHANGABLE,RATING ACTION,VALUE DATE MONTH,MATURITY DURATION,ACCRUED INTEREST,OPR movement,INFLATION RATE,MGS
0,UN190345,MYBUN1903458,GENTING RMTN MTN 3653D 08.11.2029 - Tranche 1,2020-04-01,4.543,4.448,4.353,97.196,97.920,98.650,7.684,71.122,-0.023,-0.024,-0.025,0.175,0.184,0.193,3.0,201900062,Genting RMTN Berhad,0,Fixed Rate Bond,Corporate(G),AAA (S),RAM,2019-11-08,2029-11-08,NaT,ACT365,2.0,2020-05-08,NaT,0.00,2020-05-08,4.18,10000,1000,460,460,10Y,10000,1000,NaN,TRADING & SERVICES,2019-09-18,False,False,Initial,2020-03,3653,1.660548,2.5,-0.2,3.35
1,VZ170120,MYBVZ1701201,DANAINFRA IMTN 5.100% 21.03.2042 - Tranche No 59,2020-04-01,4.485,4.372,4.259,108.536,110.211,111.922,13.574,248.751,-0.058,-0.060,-0.062,0.846,0.893,0.942,0.0,201200042,DanaInfra Nasional Berhad,1,Fixed Rate Bond,Quasi-Govt,NR(LT),NaN,2017-03-21,2042-03-21,NaT,ACT365,2.0,2017-09-21,2020-03-23,5.10,2020-09-21,5.10,61000,55820,285,285,25Y,74000,63820,MURABAHAH,FINANCIAL SERVICES,NaT,False,False,NaN,2020-03,9131,0.125753,2.5,-0.2,3.45
2,VZ170163,MYBVZ1701631,DANAINFRA IMTN 5.200% 23.05.2042 - Tranche No 66,2020-04-01,4.484,4.376,4.268,109.981,111.604,113.260,13.379,246.144,-0.058,-0.060,-0.062,0.858,0.905,0.954,1.7,201200042,DanaInfra Nasional Berhad,1,Fixed Rate Bond,Quasi-Govt,NR(LT),NaN,2017-05-25,2042-05-23,NaT,ACT365,2.0,2017-11-27,2019-11-25,5.20,2020-05-27,5.20,61000,55820,780,780,25Y,74000,63820,MURABAHAH,FINANCIAL SERVICES,NaT,False,False,NaN,2020-03,9129,1.823562,2.5,-0.2,3.45
3,VZ170377,MYBVZ1703777,DANAINFRA IMTN 5.220% 14.11.2042 - Tranche No 71,2020-04-01,4.495,4.387,4.279,110.223,111.868,113.546,13.513,252.626,-0.058,-0.060,-0.062,0.870,0.918,0.967,1.7,201200042,DanaInfra Nasional Berhad,1,Fixed Rate Bond,Quasi-Govt,NR(LT),NaN,2017-11-16,2042-11-14,NaT,ACT365,2.0,2018-05-16,2019-11-18,5.22,2020-05-18,5.22,61000,55820,600,600,25Y,74000,63820,MURABAHAH,FINANCIAL SERVICES,NaT,False,False,NaN,2020-03,9129,1.930685,2.5,-0.2,3.45
4,VZ180073,MYBVZ1800730,DANAINFRA IMTN 5.240% 20.02.2043 - Tranche No 76,2020-04-01,4.509,4.396,4.283,110.364,112.098,113.868,13.759,259.435,-0.058,-0.060,-0.062,0.875,0.925,0.975,0.0,201200042,DanaInfra Nasional Berhad,1,Fixed Rate Bond,Quasi-Govt,NR(LT),NaN,2018-02-21,2043-02-20,NaT,ACT365,2.0,2018-08-21,2020-02-21,5.24,2020-08-21,5.24,61000,55820,1000,1000,25Y,74000,63820,MURABAHAH,FINANCIAL SERVICES,NaT,False,False,NaN,2020-03,9130,0.574247,2.5,-0.2,3.45
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34201,VZ180073,MYBVZ1800730,DANAINFRA IMTN 5.240% 20.02.2043 - Tranche No 76,2020-05-04,3.895,3.781,3.667,120.194,122.153,124.156,14.101,268.667,-0.013,-0.012,-0.011,0.215,0.201,0.188,0.0,201200042,DanaInfra Nasional Berhad,1,Fixed Rate Bond,Quasi-Govt,NR(LT),NaN,2018-02-21,2043-02-20,NaT,ACT365,2.0,2018-08-21,2020-02-21,5.24,2020-08-21,5.24,61000,56820,1000,1000,25Y,74000,64820,MURABAHAH,F

In [16]:
# calculate: 
# 1. changes In EVAL MID Price 
# 2. moving average changes In EVAL MID Price for the past 5 months
# 3. NEXT MONTH CHANGES IN EVAL MID PRICE
def transform_eval_mid_price(df_joined):
  temp_df_joined = pd.DataFrame()
  for stock_code in df_joined["STOCK CODE"].unique():
    temp_df = df_joined[df_joined["STOCK CODE"] == stock_code].sort_values("VALUE DATE MONTH")
    # calculate changes In EVAL MID Price
    temp_df["CHANGES IN EVAL MID PRICE"] = temp_df["EVAL MID PRICE"].diff()
    # calculate moving average changes In EVAL MID Price for the past 5 months
    temp_df["MA-5 CHANGES IN EVAL MID PRICE"] = temp_df["CHANGES IN EVAL MID PRICE"].rolling(5,1).mean()
    # shift the NEXT MONTH CHANGES IN EVAL MID PRICE to current month 
    temp_df["NEXT MONTH CHANGES IN EVAL MID PRICE"] = temp_df["CHANGES IN EVAL MID PRICE"].shift(-1)
    temp_df_joined = pd.concat([temp_df_joined, temp_df])
    break
  return temp_df_joined

df_joined = transform_eval_mid_price(df_joined)
df_joined

,STOCK CODE,ISIN CODE,STOCK NAME,VALUE DATE,EVAL UPPER THRESHOLD YIELD,EVAL MID YIELD,EVAL LOWER THRESHOLD YIELD,EVAL LOWER THRESHOLD PRICE,EVAL MID PRICE,EVAL UPPER THRESHOLD PRICE,MODIFIED DURATION,CONVEXITY,EVAL UPPER THRESHOLD YIELD CHANGE,EVAL MID YIELD CHANGE,EVAL LOWER THRESHOLD YIELD CHANGE,EVAL LOWER THRESHOLD PRICE CHANGE,EVAL MID PRICE CHANGE,EVAL UPPER THRESHOLD PRICE CHANGE,COMPOSITE LIQUIDITY SCORE (T-1),FACILITY CODE,ISSUER NAME,PRINCIPLE,BOND TYPE,BOND CLASS,RATING,RATING AGENCY,ISSUE DATE,MATURITY DATE,EXPECTED MATURITY DATE,DAY COUNT BASIS,COUPON FREQUENCY,FIRST PAYMENT DATE,PREVIOUS PAYMENT DATE,PREVIOUS COUPON RATE,NEXT PAYMENT DATE,NEXT COUPON RATE,FACILITY AMOUNT/FACILITY LIMIT(MYR MIL),FACILITY OUTSTANDING AMOUNT(MYR MIL),BOND ISSUE AMOUNT(MYR MIL),BOND CURRENT OUTSTANDING AMOUNT(MYR MIL),REMAINING TENURE,ISSUER FACILITY LIMIT(MYR MIL),ISSUER OUTSTANDING AMOUNT(MYR MIL),ISLAMIC CONCEPT,SECTOR,RATING EFFECTIVE DATE,CALLABLE/PUTTABLE,CONVERTIBLE/EXCHANGABLE,RATING ACTION,VALUE DATE MONTH,MATURITY DURATION,ACCRUED INTEREST,OPR movement,INFLATION RATE,MGS,CHANGES IN EVAL MID PRICE,MA-5 CHANGES IN EVAL MID PRICE,NEXT MONTH CHANGES IN EVAL MID PRICE
18435,UN190345,MYBUN1903458,GENTING RMTN MTN 3653D 08.11.2029 - Tranche 1,2019-12-02,4.171,4.132,4.093,100.070,100.385,100.702,8.047,76.931,-0.002,0.000,0.002,0.016,-0.001,-0.017,2.0,201900062,Genting RMTN Berhad,0,Fixed Rate Bond,Corporate(G),AAA (S),RAM,2019-11-08,2029-11-08,NaT,ACT365,2.0,2020-05-08,NaT,0.00,2020-05-08,4.18,10000,1000,460,460,10Y,10000,1000,NaN,TRADING & SERVICES,2019-09-18,False,False,Initial,2019-11,3653,0.274849,3.00,0.9,3.42,NaN,NaN,0.012
19829,UN190345,MYBUN1903458,GENTING RMTN MTN 3653D 08.11.2029 - Tranche 1,2020-01-02,4.165,4.130,4.095,100.116,100.397,100.679,7.964,75.561,0.000,0.000,0.000,0.000,0.000,0.000,1.7,201900062,Genting RMTN Berhad,0,Fixed Rate Bond,Corporate(G),AAA (S),RAM,2019-11-08,2029-11-08,NaT,ACT365,2.0,2020-05-08,NaT,0.00,2020-05-08,4.18,10000,1000,460,460,10Y,10000,1000,NaN,TRADING & SERVICES,2019-09-18,False,False,Initial,2019-12,3653,0.629863,3.00,1.0,3.30,0.012,0.01200,2.115
28595,UN190345,MYBUN1903458,GENTING RMTN MTN 3653D 08.11.2029 - Tranche 1,2020-02-03,3.914,3.868,3.822,102.136,102.512,102.889,7.909,74.609,-0.026,-0.027,-0.028,0.210,0.219,0.228,1.7,201900062,Genting RMTN Berhad,0,Fixed Rate Bond,Corporate(G),AAA (S),RAM,2019-11-08,2029-11-08,NaT,ACT365,2.0,2020-05-08,NaT,0.00,2020-05-08,4.18,10000,1000,460,460,10Y,10000,1000,NaN,TRADING & SERVICES,2019-09-18,False,False,Initial,2020-01,3653,0.996329,2.75,1.6,3.13,2.115,1.06350,1.179
22752,UN190345,MYBUN1903458,GENTING RMTN MTN 3653D 08.11.2029 - Tranche 1,2020-03-02,3.791,3.722,3.653,103.124,103.691,104.262,7.852,73.635,-0.028,-0.029,-0.030,0.227,0.236,0.246,2.7,201900062,Genting RMTN Berhad,0,Fixed Rate Bond,Corporate(G),AAA (S),RAM,2019-11-08,2029-11-08,NaT,ACT365,2.0,2020-05-08,NaT,0.00,2020-05-08,4.18,10000,1000,460,460,10Y,10000,1000,NaN,TRADING & SERVICES,2019-09-18,False,False,Initial,2020-02,3653,1.316986,2.75,1.3,2.82,1.179,1.10200,-5.771
0,UN190345,MYBUN1903458,GENTING RMTN MTN 3653D 08.11.2029 - Tranche 1,2020-04-01,4.543,4.448,4.353,97.196,97.920,98.650,7.684,71.122,-0.023,-0.024,-0.025,0.175,0.184,0.193,3.0,201900062,Genting RMTN Berhad,0,Fixed Rate Bond,Corporate(G),AAA (S),RAM,2019-11-08,2029-11-08,NaT,ACT365,2.0,2020-05-08,NaT,0.00,2020-05-08,4.18,10000,1000,460,460,10Y,10000,1000,NaN,TRADING & SERVICES,2019-09-18,False,False,Initial,2020-03,3653,1.660548,2.50,-0.2,3.35,-5.771,-0.61625,4.743
32751,UN190345,MYBUN1903458,GENTING RMTN MTN 3653D 08.11.2029 - Tranche 1,2020-05-04,3.943,3.843,3.743,101.864,102.663,103.470,7.668,70.721,-0.040,-0.040,-0.040,0.316,0.318,0.321,3.3,201900062,Genting RMTN Berhad,0,Fixed Rate Bond,Corporate(G),AAA (S),RAM,2019-11-08,2029-11-08,NaT,ACT365,2.0,2020-05-08,NaT,0.00,2020-05-08,4.18,10000,1000,460,460,10Y,10000,1000,NaN,TRADING & SERVICES,2020-04-17,False,False,Reaffirm,2020-04,3653,2.038466,2.50,-2.9,2.89,4.743,0

In [17]:
# remove the first row and the last row of each stocks
df_joined.dropna(subset=["CHANGES IN EVAL MID PRICE","NEXT MONTH CHANGES IN EVAL MID PRICE"],inplace=True)
df_joined

,STOCK CODE,ISIN CODE,STOCK NAME,VALUE DATE,EVAL UPPER THRESHOLD YIELD,EVAL MID YIELD,EVAL LOWER THRESHOLD YIELD,EVAL LOWER THRESHOLD PRICE,EVAL MID PRICE,EVAL UPPER THRESHOLD PRICE,MODIFIED DURATION,CONVEXITY,EVAL UPPER THRESHOLD YIELD CHANGE,EVAL MID YIELD CHANGE,EVAL LOWER THRESHOLD YIELD CHANGE,EVAL LOWER THRESHOLD PRICE CHANGE,EVAL MID PRICE CHANGE,EVAL UPPER THRESHOLD PRICE CHANGE,COMPOSITE LIQUIDITY SCORE (T-1),FACILITY CODE,ISSUER NAME,PRINCIPLE,BOND TYPE,BOND CLASS,RATING,RATING AGENCY,ISSUE DATE,MATURITY DATE,EXPECTED MATURITY DATE,DAY COUNT BASIS,COUPON FREQUENCY,FIRST PAYMENT DATE,PREVIOUS PAYMENT DATE,PREVIOUS COUPON RATE,NEXT PAYMENT DATE,NEXT COUPON RATE,FACILITY AMOUNT/FACILITY LIMIT(MYR MIL),FACILITY OUTSTANDING AMOUNT(MYR MIL),BOND ISSUE AMOUNT(MYR MIL),BOND CURRENT OUTSTANDING AMOUNT(MYR MIL),REMAINING TENURE,ISSUER FACILITY LIMIT(MYR MIL),ISSUER OUTSTANDING AMOUNT(MYR MIL),ISLAMIC CONCEPT,SECTOR,RATING EFFECTIVE DATE,CALLABLE/PUTTABLE,CONVERTIBLE/EXCHANGABLE,RATING ACTION,VALUE DATE MONTH,MATURITY DURATION,ACCRUED INTEREST,OPR movement,INFLATION RATE,MGS,CHANGES IN EVAL MID PRICE,MA-5 CHANGES IN EVAL MID PRICE,NEXT MONTH CHANGES IN EVAL MID PRICE
19829,UN190345,MYBUN1903458,GENTING RMTN MTN 3653D 08.11.2029 - Tranche 1,2020-01-02,4.165,4.130,4.095,100.116,100.397,100.679,7.964,75.561,0.000,0.000,0.000,0.000,0.000,0.000,1.7,201900062,Genting RMTN Berhad,0,Fixed Rate Bond,Corporate(G),AAA (S),RAM,2019-11-08,2029-11-08,NaT,ACT365,2.0,2020-05-08,NaT,0.00,2020-05-08,4.18,10000,1000,460,460,10Y,10000,1000,NaN,TRADING & SERVICES,2019-09-18,False,False,Initial,2019-12,3653,0.629863,3.00,1.0,3.30,0.012,0.01200,2.115
28595,UN190345,MYBUN1903458,GENTING RMTN MTN 3653D 08.11.2029 - Tranche 1,2020-02-03,3.914,3.868,3.822,102.136,102.512,102.889,7.909,74.609,-0.026,-0.027,-0.028,0.210,0.219,0.228,1.7,201900062,Genting RMTN Berhad,0,Fixed Rate Bond,Corporate(G),AAA (S),RAM,2019-11-08,2029-11-08,NaT,ACT365,2.0,2020-05-08,NaT,0.00,2020-05-08,4.18,10000,1000,460,460,10Y,10000,1000,NaN,TRADING & SERVICES,2019-09-18,False,False,Initial,2020-01,3653,0.996329,2.75,1.6,3.13,2.115,1.06350,1.179
22752,UN190345,MYBUN1903458,GENTING RMTN MTN 3653D 08.11.2029 - Tranche 1,2020-03-02,3.791,3.722,3.653,103.124,103.691,104.262,7.852,73.635,-0.028,-0.029,-0.030,0.227,0.236,0.246,2.7,201900062,Genting RMTN Berhad,0,Fixed Rate Bond,Corporate(G),AAA (S),RAM,2019-11-08,2029-11-08,NaT,ACT365,2.0,2020-05-08,NaT,0.00,2020-05-08,4.18,10000,1000,460,460,10Y,10000,1000,NaN,TRADING & SERVICES,2019-09-18,False,False,Initial,2020-02,3653,1.316986,2.75,1.3,2.82,1.179,1.10200,-5.771
0,UN190345,MYBUN1903458,GENTING RMTN MTN 3653D 08.11.2029 - Tranche 1,2020-04-01,4.543,4.448,4.353,97.196,97.920,98.650,7.684,71.122,-0.023,-0.024,-0.025,0.175,0.184,0.193,3.0,201900062,Genting RMTN Berhad,0,Fixed Rate Bond,Corporate(G),AAA (S),RAM,2019-11-08,2029-11-08,NaT,ACT365,2.0,2020-05-08,NaT,0.00,2020-05-08,4.18,10000,1000,460,460,10Y,10000,1000,NaN,TRADING & SERVICES,2019-09-18,False,False,Initial,2020-03,3653,1.660548,2.50,-0.2,3.35,-5.771,-0.61625,4.743
32751,UN190345,MYBUN1903458,GENTING RMTN MTN 3653D 08.11.2029 - Tranche 1,2020-05-04,3.943,3.843,3.743,101.864,102.663,103.470,7.668,70.721,-0.040,-0.040,-0.040,0.316,0.318,0.321,3.3,201900062,Genting RMTN Berhad,0,Fixed Rate Bond,Corporate(G),AAA (S),RAM,2019-11-08,2029-11-08,NaT,ACT365,2.0,2020-05-08,NaT,0.00,2020-05-08,4.18,10000,1000,460,460,10Y,10000,1000,NaN,TRADING & SERVICES,2020-04-17,False,False,Reaffirm,2020-04,3653,2.038466,2.50,-2.9,2.89,4.743,0.45560,0.597
7806,UN190345,MYBUN1903458,GENTING RMTN MTN 3653D 08.11.2029 - Tranche 1,2020-06-01,3.887,3.766,3.645,102.293,103.260,104.237,7.755,71.049,-0.005,-0.012,-0.019,0.037,0.094,0.151,2.7,201900062,Genting RMTN Berhad,0,Fixed Rate Bond,Corporate(G),AAA (S),RAM,2019-11-08,2029-11-08,NaT,ACT365,2.0,2020-05-08,2020-05-08,4.18,2020-11-09,4.18,10000,1000,460,460,10Y,10000,1000,NaN,TRADING & SERVICES,2020-04-17,False,False,Reaffirm,2020-05,3653,0.274849,2.00,-2.